# Exploring non-nanosecond time implementations on Pandas Data Structures

In [1]:
import pathlib
import logging
import sys

import numpy as np
import pandas as pd

# Setup Logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)

logger.addHandler(handler)

logger.info(f'''
Python Version | {sys.version}
Pandas Version | {pd.__version__}
Numpy Version  | {np.__version__}
''')


Python Version | 3.10.8 (main, Nov  4 2022, 08:45:25) [Clang 12.0.0 ]
Pandas Version | 2.0.0.dev0+664.g86f182829d
Numpy Version  | 1.23.4



### pandas `date_range`

We recast the resolution on the date_range object. However, creating timestamps older than 1700-01-01 results in out of bounds

In [2]:
dr = pd.date_range('1700', periods=100).astype('datetime64[ns]')
dr[:5]

DatetimeIndex(['1700-01-01', '1700-01-02', '1700-01-03', '1700-01-04',
               '1700-01-05'],
              dtype='datetime64[ns]', freq='D')

In [3]:
dr_seconds = dr.astype('datetime64[s]')
dr_seconds[:5]

DatetimeIndex(['1700-01-01', '1700-01-02', '1700-01-03', '1700-01-04',
               '1700-01-05'],
              dtype='datetime64[s]', freq=None)

In [4]:
dr_older = pd.date_range('1500', periods=100).astype('datetime64[ns]')

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1500-01-01 00:00:00

### pandas `Series`

See above; same result

In [5]:
seconds = pd.Series(['1700-01-01'], dtype='datetime64[s]')
nanoseconds = pd.Series(['1700-01-01'], dtype='datetime64[ns]')

seconds, nanoseconds

(0   1700-01-01
 dtype: datetime64[s],
 0   1700-01-01
 dtype: datetime64[ns])

In [6]:
seconds = pd.Series(['1500-01-01'], dtype='datetime64[s]')

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1500-01-01 00:00:00 present at position 0

### pandas hacking

Let's see what happens when we try and create a range of datetimes represented in nano-seconds, which when changing "resolution" should include new data, and then see if we can access them.

[PR 47675](https://github.com/pandas-dev/pandas/pull/47675/files) - datetime_scalar support non-nano

Helpful Links:
* [NumPy DateTime Units](https://numpy.org/doc/stable/reference/arrays.datetime.html#datetime-units)
* [Original Proposal](https://numpy.org/doc/1.13/neps/datetime-proposal.html)

I *think* this means I'll have to roll a custom datetime class, and then wrap that into the pandas extension.